In [1]:
hf_dataset_identifier = "segments/sidewalk-semantic"

In [36]:
from datasets import load_dataset

ds = load_dataset(hf_dataset_identifier)

Using custom data configuration segments--sidewalk-semantic-2-1680095ec7b2b03f
Found cached dataset parquet (/home/shinichi/.cache/huggingface/datasets/segments___parquet/segments--sidewalk-semantic-2-1680095ec7b2b03f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
ds

DatasetDict({
    train: Dataset({
        features: ['pixel_values', 'label'],
        num_rows: 1000
    })
})

In [9]:
ds = ds.shuffle(seed=42)
ds = ds["train"].train_test_split(test_size=0.2)
train_ds = ds["train"]
test_ds = ds["test"]



Loading cached shuffled indices for dataset at /home/shinichi/.cache/huggingface/datasets/segments___parquet/segments--sidewalk-semantic-2-1680095ec7b2b03f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-58b6aa6a2f0a0d6b.arrow


In [11]:
import json
from huggingface_hub import hf_hub_download

repo_id = f"datasets/{hf_dataset_identifier}"
filename = "id2label.json"
id2label = json.load(open(hf_hub_download(repo_id=hf_dataset_identifier, filename=filename, repo_type="dataset"), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)


In [12]:
id2label

{0: 'unlabeled',
 1: 'flat-road',
 2: 'flat-sidewalk',
 3: 'flat-crosswalk',
 4: 'flat-cyclinglane',
 5: 'flat-parkingdriveway',
 6: 'flat-railtrack',
 7: 'flat-curb',
 8: 'human-person',
 9: 'human-rider',
 10: 'vehicle-car',
 11: 'vehicle-truck',
 12: 'vehicle-bus',
 13: 'vehicle-tramtrain',
 14: 'vehicle-motorcycle',
 15: 'vehicle-bicycle',
 16: 'vehicle-caravan',
 17: 'vehicle-cartrailer',
 18: 'construction-building',
 19: 'construction-door',
 20: 'construction-wall',
 21: 'construction-fenceguardrail',
 22: 'construction-bridge',
 23: 'construction-tunnel',
 24: 'construction-stairs',
 25: 'object-pole',
 26: 'object-trafficsign',
 27: 'object-trafficlight',
 28: 'nature-vegetation',
 29: 'nature-terrain',
 30: 'sky',
 31: 'void-ground',
 32: 'void-dynamic',
 33: 'void-static',
 34: 'void-unclear'}

In [13]:
label2id

{'unlabeled': 0,
 'flat-road': 1,
 'flat-sidewalk': 2,
 'flat-crosswalk': 3,
 'flat-cyclinglane': 4,
 'flat-parkingdriveway': 5,
 'flat-railtrack': 6,
 'flat-curb': 7,
 'human-person': 8,
 'human-rider': 9,
 'vehicle-car': 10,
 'vehicle-truck': 11,
 'vehicle-bus': 12,
 'vehicle-tramtrain': 13,
 'vehicle-motorcycle': 14,
 'vehicle-bicycle': 15,
 'vehicle-caravan': 16,
 'vehicle-cartrailer': 17,
 'construction-building': 18,
 'construction-door': 19,
 'construction-wall': 20,
 'construction-fenceguardrail': 21,
 'construction-bridge': 22,
 'construction-tunnel': 23,
 'construction-stairs': 24,
 'object-pole': 25,
 'object-trafficsign': 26,
 'object-trafficlight': 27,
 'nature-vegetation': 28,
 'nature-terrain': 29,
 'sky': 30,
 'void-ground': 31,
 'void-dynamic': 32,
 'void-static': 33,
 'void-unclear': 34}

In [14]:
from torchvision.transforms import ColorJitter
from transformers import SegformerFeatureExtractor

feature_extractor = SegformerFeatureExtractor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = feature_extractor(images, labels)
    return inputs


# Set transforms
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)


2022-12-14 12:36:27.538841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-14 12:36:27.741070: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-14 12:36:27.823488: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-14 12:36:28.566662: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

['pixel_values', 'label']

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

TypeError: img should be PIL Image. Got <class 'dict'>

In [7]:
from transformers import SegformerForSemanticSegmentation

pretrained_model_name = "nvidia/mit-b0" 
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)


Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.linear_fuse.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.weight', 'decode_head.classifier.

In [8]:
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    # currently using _compute instead of compute
    # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
#             reduce_labels=feature_extractor.reduce_labels,
        )
    
    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})
    
    return metrics


In [9]:
from transformers import TrainingArguments

epochs = 10
lr = 0.00006
batch_size = 8

hub_model_id = "segformer-b0-finetuned-segments-sidewalk-2"

training_args = TrainingArguments(
    "segformer-b0-finetuned-segments-sidewalk-outputs",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    hub_strategy="end",
)


In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)


In [11]:
trainer.train()


/home/shinichi/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 800
  Num Epochs = 50
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5000
  Number of trainable parameters = 3723139


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Unlabeled,Accuracy Flat-road,Accuracy Flat-sidewalk,Accuracy Flat-crosswalk,Accuracy Flat-cyclinglane,Accuracy Flat-parkingdriveway,Accuracy Flat-railtrack,Accuracy Flat-curb,Accuracy Human-person,Accuracy Human-rider,Accuracy Vehicle-car,Accuracy Vehicle-truck,Accuracy Vehicle-bus,Accuracy Vehicle-tramtrain,Accuracy Vehicle-motorcycle,Accuracy Vehicle-bicycle,Accuracy Vehicle-caravan,Accuracy Vehicle-cartrailer,Accuracy Construction-building,Accuracy Construction-door,Accuracy Construction-wall,Accuracy Construction-fenceguardrail,Accuracy Construction-bridge,Accuracy Construction-tunnel,Accuracy Construction-stairs,Accuracy Object-pole,Accuracy Object-trafficsign,Accuracy Object-trafficlight,Accuracy Nature-vegetation,Accuracy Nature-terrain,Accuracy Sky,Accuracy Void-ground,Accuracy Void-dynamic,Accuracy Void-static,Accuracy Void-unclear,Iou Unlabeled,Iou Flat-road,Iou Flat-sidewalk,Iou Flat-crosswalk,Iou Flat-cyclinglane,Iou Flat-parkingdriveway,Iou Flat-railtrack,Iou Flat-curb,Iou Human-person,Iou Human-rider,Iou Vehicle-car,Iou Vehicle-truck,Iou Vehicle-bus,Iou Vehicle-tramtrain,Iou Vehicle-motorcycle,Iou Vehicle-bicycle,Iou Vehicle-caravan,Iou Vehicle-cartrailer,Iou Construction-building,Iou Construction-door,Iou Construction-wall,Iou Construction-fenceguardrail,Iou Construction-bridge,Iou Construction-tunnel,Iou Construction-stairs,Iou Object-pole,Iou Object-trafficsign,Iou Object-trafficlight,Iou Nature-vegetation,Iou Nature-terrain,Iou Sky,Iou Void-ground,Iou Void-dynamic,Iou Void-static,Iou Void-unclear
20,2.695300,3.036437,0.085674,0.130929,0.616076,nan,0.368819,0.939261,0.000696,0.011123,0.004210,0.000000,0.002869,0.000000,0.000388,0.646413,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.938764,0.000000,0.000099,0.003537,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.856654,0.059455,0.488303,0.000000,0.000000,0.000068,0.000000,0.000000,0.318711,0.661635,0.000686,0.011055,0.003832,0.000000,0.002771,0.000000,0.000144,0.509603,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.361207,0.000000,0.000099,0.003219,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.605909,0.055081,0.464571,0.000000,0.000000,0.000067,0.000000
40,2.233500,2.169182,0.103330,0.149557,0.657690,nan,0.727978,0.899967,0.000019,0.001620,0.002193,0.000000,0.000122,0.000000,0.000000,0.791398,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.908660,0.000000,0.000000,0.000052,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.927032,0.013510,0.662822,0.000000,0.000000,0.000019,0.000000,nan,0.420034,0.732455,0.000019,0.001618,0.002150,0.000000,0.000122,0.000000,0.000000,0.569694,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.444495,0.000000,0.000000,0.000052,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.606438,0.012922,0.619873,0.000000,0.000000,0.000019,0.000000
60,1.789900,1.816899,0.112035,0.159599,0.676099,nan,0.798541,0.903067,0.000009,0.004025,0.000718,0.000000,0.000053,0.000000,0.000000,0.859942,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.878256,0.000000,0.000000,0.000000,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.933503,0.091766,0.796874,0.000000,0.000000,0.000007,0.000000,nan,0.424962,0.748934,0.000009,0.004024,0.000714,0.000000,0.000053,0.000000,0.000000,0.569267,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.489418,0.000000,0.000000,0.000000,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.642057,0.085775,0.731920,0.000000,0.000000,0.000007,0.000000
80,1.922600,1.657836,0.120091,0.167319,0.691423,nan,0.839969,0.906810,0.000000,0.039570,0.000424,0.000000,0.000031,0.000000,0.000000,0.867455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.852279,0.000000,0.000000,0.000000,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.948006,0.216690,0.850293,0.000000,0.000000,0.000000,0.000000,nan,0.448072,0.762164,0.000000,0.039529

***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
Saving model checkpoint to segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-20
Configuration saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-20/config.json
Model weights saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-20/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-480] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259:

Saving model checkpoint to segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-180
Configuration saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-180/config.json
Model weights saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-180/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-120] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total

Saving model checkpoint to segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-340
Configuration saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-340/config.json
Model weights saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-340/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-280] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total

...

Saving model checkpoint to segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1140
Configuration saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1140/config.json
Model weights saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1140/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1080] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = t

Saving model checkpoint to segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1300
Configuration saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1300/config.json
Model weights saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1300/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1260] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = t

Saving model checkpoint to segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1460
Configuration saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1460/config.json
Model weights saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1460/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1400] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = t

Saving model checkpoint to segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1620
Configuration saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1620/config.json
Model weights saved in segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1620/pytorch_model.bin
Deleting older checkpoint [segformer-b0-finetuned-segments-sidewalk-outputs/checkpoint-1460] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/home/shinichi/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = t

KeyboardInterrupt: 

SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  